
# EDA and regression using Sklearn (Boston Dataset)

Prepared by:

<b> Noman H chowdhury </b>

Reference for theoretical content:

Chapater 3 ISLP https://www.statlearning.com/


### Regression - only SKlearn with EDA (Boston dataset)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# test line to chk github


In [ ]:
# Ignore

We  will use the `Boston` housing data set.  The `Boston` dataset records  `medv`  (median house value) for $506$ neighborhoods
around Boston.  We will build a regression model to predict  `medv`  using $13$
predictors such as  `rmvar`  (average number of rooms per house),
 `age`  (proportion of owner-occupied units built prior to 1940), and  `lstat`  (percent of
households with low socioeconomic status).  We will use `statsmodels` for this
task, a `Python` package that implements several commonly used
regression methods.

In [ ]:
import os
os.chdir("path of working directory")
os.getcwd()
#

In [3]:
Boston = pd.read_csv("Boston.csv")

In [ ]:
# Check for missing values in each column
missing_values = Boston.isnull().sum()
print(missing_values)

In [ ]:
# Check the percentage of missing values for each column
missing_percentage = Boston.isnull().mean() * 100
print(missing_percentage)

In [10]:
# Fill missing numerical values with the mean
Boston['lstat'] = Boston['lstat'].fillna(Boston['lstat'].mean())

# For categorical data, you might use the mode
# Boston['categorical_column'] = Boston['categorical_column'].fillna(Boston['categorical_column'].mode()[0])

In [12]:
# We can also Drop columns with a high percentage of missing values
# Boston = Boston.drop(columns=['column_to_drop'])

# Drop rows with any missing values
Boston = Boston.dropna()

In [ ]:
### Basic Statistics
print(Boston.describe())

In [ ]:
# Histogram for numerical features
Boston.hist(bins=15, figsize=(15, 10))
plt.suptitle("Histograms of Numerical Features")
plt.show()

In [ ]:
# Boxplot for numerical features to identify outliers
Boston.plot(kind='box', subplots=True, layout=(4,4), figsize=(15, 10))
plt.suptitle("Boxplots of Numerical Features")
plt.show()

In [20]:
# Outlier handling Example: Remove outliers beyond 3 standard deviations from the mean
from scipy import stats
Boston_cleaned = Boston[(np.abs(stats.zscore(Boston)) < 3).all(axis=1)]

In [ ]:
# Correlation matrix
correlation_matrix = Boston.corr()
correlation_matrix

In [ ]:
import seaborn as sns
# Heatmap visualization of the correlation matrix
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
plt.title("Correlation Matrix of Features")
plt.show()

In [ ]:
### Pair Plot for Selected Features

# Selecting a subset of features for pair plot
subset_features = ['rm', 'age', 'lstat', 'medv']
#subset_features = ['medv', 'lstat', 'rm',\
#    'ptratio', 'age']

# Pair plot
#sns.pairplot(Boston[subset_features])
sns.pairplot(Boston[subset_features], kind='reg')
plt.suptitle("Pair Plot of Selected Features")
plt.show()

In [29]:
# Correlation with the target variable 'medv'
correlation_with_target = Boston.corr()['medv'].sort_values(ascending=False)
print("Correlation with the target variable (medv):")
print(correlation_with_target)

Correlation with the target variable (medv):
medv       1.000000
rm         0.695360
zn         0.360445
dis        0.249929
chas       0.175260
age       -0.376955
rad       -0.381626
crim      -0.388305
nox       -0.427321
tax       -0.468536
indus     -0.483725
ptratio   -0.507787
lstat     -0.737231
Name: medv, dtype: float64


### Profiling - automatic

In [ ]:
!pip install ydata-profiling

We can use the ydata-profiling library, which is a powerful tool for generating comprehensive reports on your data, including statistics, correlations, and even the identification of missing values. It's particularly useful at the exploratory data analysis (EDA) stage of your project because it provides a quick and deep insight into the dataset with minimal code. This can help you make informed decisions on how to handle missing data, among other preprocessing steps.

In [ ]:
from ydata_profiling import ProfileReport

# Generate the profile report
profile = ProfileReport(Boston, title='Pandas Profiling Report', explorative=True)

# To display the report in a Jupyter notebook
profile.to_notebook_iframe()

# To save the report to a file
# profile.to_file("boston_data_profile.html")

If you're working with particularly large datasets, pandas-profiling can be resource-intensive and may take some time to generate the report. In such cases, it's possible to customize the report generation to skip certain analyses or to increase efficiency by adjusting the minimal=True parameter in the ProfileReport function.

## Now, Regression with sklearn

In scikit-learn, there isn't a built-in method equivalent to the summary output you might be familiar with from statistical packages like R's lm() function or Python's statsmodels. However, you can manually compute and display key statistics for your regression model, such as R-squared, adjusted R-squared, coefficients, standard errors, p-values, and the confidence intervals for the coefficients.

For a simple linear regression model using scikit-learn, you can manually calculate and display some of these statistics, but for a more detailed summary, including p-values and confidence intervals, you would typically need to use the statsmodels library.

Here's a basic example of how to get a summary of key statistics from a scikit-learn linear regression model, followed by a more detailed example using statsmodels:

In [ ]:
!pip install scikit-learn

In [6]:
# Using scikit-learn to Display Basic Model Summaryimport pandas as pd
# Interesting that installation name and import name are different
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression # we could also use sklearn.LinearRegression (??)
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Select your features and target

X = Boston[['lstat']] # or X = Boston['lstat'].values.reshape(-1,1); # double brackets for X to be a dataframe
Y = Boston['medv']
# Optional: Split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit the regression model
model = LinearRegression()
model.fit(X, Y)
#print(model_fitted_sk.summary()) # This will not work as there is no summary method in sklearn; whereas statsmodels has a summary method (print(model_fitted_sm.summary() works)
print("Model coefs:", model.coef_) # however we can get the coefficients
# print(f'Coefficient for lstat: {model.coef_[0]}') # and the coefficient for lstat
print("Model Intercept: ",model.intercept_) # and the intercept

#print(model_fitted_sk.summary()) # this works for statsmodels but not for sklearn

# Predictions and evaluation
Y_pred = model.predict(X) # or X_train
mse = mean_squared_error(Y, Y_pred) # or y_train
print(f'Mean Squared Error: {mse}')
#print("R-squared:", model.score(X, Y)) # or X_train, y_train
print("R-squared:", r2_score(Y,Y_pred)) # CAUTION: This is the same as the previous line but they take different arguments (Y, Y_pred) vs. (X, Y)
print("Adjusted R-squared:", 1 - (1-model.score(X, Y))*(len(Y)-1)/(len(Y)-X.shape[1]-1)) # or X_train, y_train

# Calculate residuals
residuals = Y - Y_pred

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(Y_pred, residuals, color='blue', s=50, alpha=0.6)
plt.hlines(y=0, xmin=min(Y_pred), xmax=max(Y_pred), colors='red', linestyles='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Fitted Values vs. Residuals: Model 0')
plt.show()

In [ ]:
# Model 1 with training and test data with only one feature

X = Boston[['lstat']] # or X = Boston['lstat'].values.reshape(-1,1); # double brackets for X to be a dataframe
Y = Boston['medv'] # or Y = Boston.medv, did not put in double brackets as it is a series

# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Create and fit the regression model on the training set
model = LinearRegression()
model.fit(X_train, Y_train)

# Make predictions on both the training and test sets
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

# Calculate and print the MSE for both
mse_train = mean_squared_error(Y_train, Y_train_pred)
mse_test = mean_squared_error(Y_test, Y_test_pred)

# Calculate and print the R-squared for both
r2_train = r2_score(Y_train, Y_train_pred)
r2_test = r2_score(Y_test, Y_test_pred)

print(f'Model 1 - Training MSE: {mse_train}, Test MSE: {mse_test}')
print(f'Model 1 - Training R^2: {r2_train}, Test R^2: {r2_test}')

# Print the model coefficients and intercept
print(f'Intercept: {model.intercept_}')
print(f'Coefficient for lstat: {model.coef_[0]}')
print('All Coefficients:', model.coef_)

# Calculate residuals
residuals = Y_train - Y_train_pred

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(Y_train_pred, residuals, color='blue', s=50, alpha=0.6)
plt.hlines(y=0, xmin=min(Y_train_pred), xmax=max(Y_train_pred), colors='red', linestyles='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Fitted Values vs. Residuals: Model 1')
plt.show()


## Multiple Linear Regression


In [ ]:
# Model 2: Including 'lstat' and 'age'

X = Boston[['lstat', 'age']]
Y = Boston['medv'] # or Y = Boston.medv, did not put in double brackets as it is a series

In [ ]:
# Model 3: Using all available predictors

X = Boston.drop('medv', axis=1)  # Drop the target to use all other columns as predictors
Y = Boston['medv']

## Non-linear Transformations of the Predictors

In [ ]:
# Model 4: Add lstat^2 to the dataset

Boston['lstat_squared'] = Boston['lstat']**2
X = Boston[['lstat', 'lstat_squared']]
Y = Boston['medv']

In [ ]:
# Model 5: Add lstat^2 and age to the dataset

Boston['lstat_squared'] = Boston['lstat']**2
X = Boston[['lstat', 'lstat_squared', 'age']]
Y = Boston['medv']

In [ ]:
# OPTIONAL -> Model 6 (OPTIONAL if you are not comfortable with interaction terms): Add an interaction term between 'lstat' and 'age'

# Create the interaction term and add it to the dataset

#Boston['lstat_squared'] = Boston['lstat']**2
Boston['lstat_age_interaction'] = Boston['lstat'] * Boston['age']
X = Boston[['lstat', 'age', 'lstat_age_interaction']]
Y = Boston['medv']

## Qualitative Predictors


In [ ]:
# Model 7: Using 'lstat' and 'chas'
X = Boston[['lstat', 'chas']]
Y = Boston['medv']

In [ ]:
# OPTIONAL -> Model 8: Using 'age' and one-hot encoded 'lstat' categories (OPTIONAL: just for demonstration how to convert a categorical variable to numerical)

# Define bins and labels
bins = [0, 15, 30, float('inf')]
labels = ['low', 'medium', 'high']

# Bin the 'lstat' variable
Boston['lstat_cat'] = pd.cut(Boston['lstat'], bins=bins, labels=labels, right=False)

# Check the new categorical variable
print(Boston['lstat_cat'].value_counts())


# One-hot encode 'lstat_cat'
lstat_dummies = pd.get_dummies(Boston['lstat_cat'], prefix='lstat')

# Join the encoded DataFrame with the original DataFrame (ensure to avoid duplicating the 'medv' column)
Boston_encoded = Boston.join(lstat_dummies).drop(['lstat_cat'], axis=1)

# Preview to ensure encoding worked as expected
print(Boston_encoded.head())

# Select predictors - 'age' and one-hot encoded 'lstat' categories, excluding one category
X = Boston_encoded[['age', 'lstat_medium', 'lstat_high']]  # Dropped 'lstat_low'
Y = Boston_encoded['medv']

## KNN


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming 'Boston' DataFrame is already loaded
X = Boston[['lstat']]
Y = Boston['medv']

# For some models like KNN, we need to do feature scaling
scaler = StandardScaler(with_mean=True, with_std=True, copy=True)
scaler.fit(X)
X_std = scaler.transform(X)


# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X_std, Y, test_size=0.2, random_state=42)

# Create and fit the KNN regression model on the training set
# Let's arbitrarily choose K=5
k = 5
model_knn = KNeighborsRegressor(n_neighbors=k)
model_knn.fit(X_train, Y_train)

# Make predictions on both the training and test sets
Y_train_pred = model_knn.predict(X_train)
Y_test_pred = model_knn.predict(X_test)

# Calculate and print the MSE for both
mse_train_knn = mean_squared_error(Y_train, Y_train_pred)
mse_test_knn = mean_squared_error(Y_test, Y_test_pred)

# Calculate and print the R-squared for both
r2_train_knn = r2_score(Y_train, Y_train_pred)
r2_test_knn = r2_score(Y_test, Y_test_pred)

print(f'Model KNN with k={k} - Training MSE: {mse_train_knn}, Test MSE: {mse_test_knn}')
print(f'Model KNN with k={k} - Training R^2: {r2_train_knn}, Test R^2: {r2_test_knn}')

# Calculate residuals for training data to plot
residuals_knn = Y_train - Y_train_pred

# Plot for KNN Model
plt.figure(figsize=(10, 6))
plt.scatter(Y_train_pred, residuals_knn, color='blue', s=50, alpha=0.6)
plt.hlines(y=0, xmin=min(Y_train_pred), xmax=max(Y_train_pred), colors='red', linestyles='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title(f'Fitted Values vs. Residuals: Model KNN with k={k}')
plt.show()
